In [ ]:
import pandas as pd
import plotly.express as px
import argparse
import pathlib
from datetime import datetime

path = pathlib.Path().absolute()
path = str(path).replace("src/drm","")
dossier_graphes=path+"/graphes/"
csv = path+"/data/contrats/export_bi_contrats.csv"  #il manque un ; à la fin du header.
source = "DRM Inter-Rhône"

sort_week = [31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]


In [ ]:
#arguments
id_operateur=None

parser = argparse.ArgumentParser()
parser.add_argument("id_operateur", help="Identifiant opérateur", default=id_operateur, nargs='?')

try:
   args = parser.parse_args()
   id_operateur = args.id_operateur
except:
   print("Arguments pas défaut")

In [ ]:
contrats = pd.read_csv(csv, sep=";",encoding="iso8859_15", low_memory=False)

contrats['date de validation'] = pd.to_datetime(contrats['date de validation'], utc=True)
contrats['semaine'] = contrats['date de validation'].dt.isocalendar().week
#contrats['semaine'].unique()
lastcampagnes = contrats['campagne'].unique()
lastcampagnes.sort()
lastcampagnes = lastcampagnes[-5:]


contrats = contrats.query('campagne in @lastcampagnes')
contrats['couleur'] = contrats['couleur'].str.upper()

contrats.rename(columns = {'identifiant vendeur':'identifiant_vendeur','volume enleve (en hl)':'volume enleve'}, inplace = True)

if(id_operateur):
    contrats = contrats.query("identifiant_vendeur == @id_operateur").reset_index()

# PAR APPELLATION ET COULEUR

contrats['filtre_produit'] = contrats['appellation'] + "-" + contrats['lieu'] + "-" +contrats['certification']+ "-" +contrats['genre']+ "-" +contrats['mention']

contrats_spe_spe = contrats.groupby(["identifiant_vendeur","filtre_produit", "couleur","campagne","semaine"]).sum(["volume enleve"])[["volume enleve"]]
contrats_spe_spe.reset_index(level=['semaine'], inplace=True)
contrats_spe_spe['semaine-sort'] = (contrats_spe_spe['semaine']-31)%53
contrats_spe_spe = contrats_spe_spe.sort_values(by=["identifiant_vendeur","filtre_produit", "couleur","campagne",'semaine-sort'])
contrats_spe_spe['volume'] = contrats_spe_spe.groupby(["identifiant_vendeur","filtre_produit", "couleur","campagne"])['volume enleve'].cumsum()
contrats_spe_spe = contrats_spe_spe.reset_index()
contrats_spe_spe.set_index(['identifiant_vendeur','filtre_produit','couleur'], inplace = True)

#contrats_spe_spe


# PAR APPELLATIONS

contrats_spe_all = contrats.groupby(["identifiant_vendeur","filtre_produit", "campagne","semaine"]).sum(["volume enleve"])[["volume enleve"]]
contrats_spe_all.reset_index(level=['semaine'], inplace=True)
contrats_spe_all['semaine-sort'] = (contrats_spe_all['semaine']-31)%53
contrats_spe_all = contrats_spe_all.sort_values(by=["identifiant_vendeur","filtre_produit","campagne",'semaine-sort'])
contrats_spe_all['volume'] = contrats_spe_all.groupby(["identifiant_vendeur","filtre_produit","campagne"])['volume enleve'].cumsum()
contrats_spe_all["couleur"] = "TOUT"
contrats_spe_all = contrats_spe_all.reset_index()
contrats_spe_all.set_index(['identifiant_vendeur','filtre_produit','couleur'], inplace = True)
#contrats_spe_all


#AUCUN FILTRE TOUTES LES APPELLATIONS ET TOUTES LES COULEURS

contrats_all_all = contrats.groupby(["identifiant_vendeur","campagne","semaine"]).sum(["volume enleve"])[["volume enleve"]]
contrats_all_all.reset_index(level=['semaine'], inplace=True)
contrats_all_all['semaine-sort'] = (contrats_all_all['semaine']-31)%53
contrats_all_all = contrats_all_all.sort_values(by=["identifiant_vendeur","campagne",'semaine-sort'])
contrats_all_all['volume'] = contrats_all_all.groupby(["identifiant_vendeur","campagne"])['volume enleve'].cumsum()
contrats_all_all["couleur"] = "TOUT"
contrats_all_all["filtre_produit"] = "TOUT"
contrats_all_all = contrats_all_all.reset_index()
contrats_all_all.set_index(['identifiant_vendeur','filtre_produit','couleur'], inplace = True)
contrats_all_all


#CONCATENATION DES 3 TABLEAUX :
df_final = pd.concat([contrats_spe_spe, contrats_spe_all])
df_final = pd.concat([df_final, contrats_all_all])
df_final = df_final.sort_values(by=['identifiant_vendeur', 'filtre_produit','couleur'])

df_final

In [ ]:
def create_graphe(df,identifiant,appellation,couleur):
    fig = px.line(df, x="semaine", y="volume", color='campagne')
    fig.update_layout(xaxis_type = 'category')
    fig.update_xaxes(categoryorder='array', categoryarray= sort_week)
    #fig.show()

    dossier = dossier_graphes+"/"+identifiant+"/contrat/"+appellation+"-"+couleur
    pathlib.Path(dossier).mkdir(parents=True, exist_ok=True)

    fig.write_html(dossier+"/graphe3.html",include_plotlyjs=True)

    return

In [ ]:
for bloc in df_final.index.unique():
    df = df_final.loc[[bloc]]
    create_graphe(df,bloc[0],bloc[1],bloc[2])